In [2]:
import numpy as np
import xarray as xr
from pyhdf.SD import SD, SDC

In [3]:
import glob
import os

In [4]:
# Define the directory path
directory_path = '/home/data/ywang/modis-dsr-2022/'

# Define the pattern to match files
pattern = "MCD18C1.A*.061.*"

# Create the full path with pattern
full_path_pattern = os.path.join(directory_path, pattern)

# Use glob to find all files matching the pattern
matching_files = glob.glob(full_path_pattern)
matching_files.sort()
dsrl = matching_files[:10]

In [5]:
# Define the directory path
directory_path = '/home/data/ywang/modis-par-2022/'

# Define the pattern to match files
pattern = "MCD18C1.A*.061.*"

# Create the full path with pattern
full_path_pattern = os.path.join(directory_path, pattern)

# Use glob to find all files matching the pattern
matching_files = glob.glob(full_path_pattern)
matching_files.sort()
parl = matching_files[:10]

In [6]:
def calculate_zoom_factors(lat_A, lon_A, lat_B, lon_B):
    """Calculate zoom factors for latitude and longitude dimensions."""
    zoom_factor_lat = len(lat_A) / len(lat_B)
    zoom_factor_lon = len(lon_A) / len(lon_B)
    return (zoom_factor_lat, zoom_factor_lon)

In [7]:
from scipy.ndimage import zoom

def resample_dataset_B(data_B, zoom_factors):
    """Resample Dataset B spatially to match Dataset A's resolution."""
    resampled_B = np.empty((data_B.shape[0], int(data_B.shape[1] * zoom_factors[0]), int(data_B.shape[2] * zoom_factors[1])))
    for i in range(data_B.shape[0]):
        resampled_B[i] = zoom(data_B[i], zoom_factors, order=1)  # Using bilinear interpolation (order=1)
    return resampled_B

In [8]:
lat_A = np.arange(90, -90.25, -0.25)
lon_A = np.arange(0, 360, 0.25)
lat_B = np.arange(90, -90, -0.05)
lon_B = np.arange(-180, 180, 0.05)

# Convert longitudes from -180 to 180 range to 0 to 360 range
lon_B_t = np.mod(lon_B + 360, 360)

# Get the indices that would sort the longitude array
sorted_indices = np.argsort(lon_B_t)

# Use these indices to sort both the longitude array and the target array
lon_B_adj = lon_B_t[sorted_indices]

# Assuming lat_A and lon_A are defined with your desired target resolution and extent
zoom_factors = calculate_zoom_factors(lat_A, lon_A, lat_B, lon_B)

In [9]:
def read_hdf_variables(file_name):
    hdf = SD(file_name, SDC.READ)
    vars_data = []
    for ds_name in hdf.datasets().keys():
        dataset = hdf.select(ds_name)
        data = dataset.get()
        data = np.where(data == -1, np.nan, data)
        vars_data.append(data)
    hdf.end()
    vars_data = np.array(vars_data)
    return vars_data

In [10]:
def remap_rad(data):
    dataa = data[:, :, sorted_indices]
    datar = resample_dataset_B(dataa, zoom_factors)
    return datar

In [12]:
dsr = []

for file_path in dsrl:
    array = read_hdf_variables(file_path)
    dsrr = remap_rad(array)
    dsr.append(dsrr)

dsr = np.concatenate(dsr, axis=0)

/home/data/ywang/modis-dsr-2022/MCD18C1.A2022001.061.2022058062929.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022002.061.2022058063131.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022003.061.2022058063737.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022004.061.2022058063838.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022005.061.2022058063939.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022006.061.2022059143939.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022007.061.2022059152121.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022008.061.2022059152828.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022009.061.2022059154646.hdf
/home/data/ywang/modis-dsr-2022/MCD18C1.A2022010.061.2022059163737.hdf


In [13]:
par = []

for file_path in dsrl:
    array = read_hdf_variables(file_path)
    parr = remap_rad(array)
    par.append(parr)

par = np.concatenate(par, axis=0)

In [16]:
era = xr.open_dataset('/home/data/ywang/adaptor.mars.internal-1713218677.3867846-29785-15-4cc9d748-57e1-4a66-ac75-1a459ea0d3f4.nc')

In [18]:
def read_nc_variables(variable_name):
    variable = era[variable_name]
    scale_factor = variable.attrs.get('scale_factor', 1)
    add_offset = variable.attrs.get('add_offset', 0)
    missing_value = variable.attrs.get('missing_value', None)
    _fill_value = variable.attrs.get('_FillValue', None)
    adjusted_variable = variable * scale_factor + add_offset
    if missing_value is not None:
        adjusted_variable = adjusted_variable.where(adjusted_variable != missing_value, other=np.nan)
    if _fill_value is not None:
        adjusted_variable = adjusted_variable.where(adjusted_variable != _fill_value, other=np.nan)
    return adjusted_variable

In [19]:
fal = read_nc_variables('fal').values
tcc = read_nc_variables('tcc').values
tcw = read_nc_variables('tcw').values
tco3 = read_nc_variables('tco3').values

In [42]:
X = np.stack([dsr, fal, tcc, tcw, tco3], axis=-1)
y = par[..., np.newaxis]

In [44]:
np.save('X.npy', X)
np.sace('y.npy', y)

AttributeError: module 'numpy' has no attribute 'sace'

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [41]:
# Stack arrays together to make handling easier
X = np.stack([dsr, fal, tcc, tcw, tco3], axis=-1)
y = par[..., np.newaxis]

# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Reshape data for scaling
n_train, lat, lon, channels = X_train.shape
X_train_flat = X_train.reshape(-1, channels)
X_val_flat = X_val.reshape(-1, channels)
X_test_flat = X_test.reshape(-1, channels)
y_train_flat = y_train.reshape(-1, 1)
y_val_flat = y_val.reshape(-1, 1)
y_test_flat = y_test.reshape(-1, 1)

# Initialize and fit the StandardScaler on the training data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_flat)
y_train_scaled = scaler_y.fit_transform(y_train_flat)

# Transform validation and test data with the same scaler
X_val_scaled = scaler_X.transform(X_val_flat)
X_test_scaled = scaler_X.transform(X_test_flat)
y_val_scaled = scaler_y.transform(y_val_flat)
y_test_scaled = scaler_y.transform(y_test_flat)

# Reshape back to the original dimensions
X_train = X_train_scaled.reshape(n_train, lat, lon, channels)
X_val = X_val_scaled.reshape(len(X_val), lat, lon, channels)
X_test = X_test_scaled.reshape(len(X_test), lat, lon, channels)
y_train = y_train_scaled.reshape(n_train, lat, lon, 1)
y_val = y_val_scaled.reshape(len(y_val), lat, lon, 1)
y_test = y_test_scaled.reshape(len(y_test), lat, lon, 1)

KeyboardInterrupt: 